In [1]:
#= Importy i załadowanie danych =#
using JLD2
using Printf
using Statistics
using LinearAlgebra

include("src/AD.jl")
include("src/NN.jl")

Main.NN

In [2]:
#= Przygotowanie danych =#
file = load("data/imdb_dataset_prepared.jld2")
X_train = Int.(file["X_train"])
y_train = vec(Float32.(file["y_train"]))
X_test  = Int.(file["X_test"])
y_test  = vec(Float32.(file["y_test"]))
embeddings = file["embeddings"]
vocab = file["vocab"]
embedding_dim = size(embeddings, 1)

50

In [3]:
dataset = NN.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

Main.NN.DataLoader((Float32[6391.0 286.0 … 11382.0 8311.0; 143.0 286.0 … 4254.0 2891.0; … ; 12849.0 12849.0 … 12849.0 12849.0; 12849.0 12849.0 … 12849.0 12849.0], Float32[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]), 64, true, [19964, 5402, 35361, 23755, 17553, 19461, 22961, 37269, 24408, 21520  …  11063, 24113, 37410, 23679, 35494, 31630, 3628, 3882, 15117, 36928])

In [4]:
model = NN.Chain(
    x -> Int.(x),
    NN.Embedding(length(vocab), embedding_dim, embeddings),
    x -> permutedims(x, (2, 1, 3)),
    NN.Conv1D(3, embedding_dim, 8, AD.relu),
    NN.MaxPool1D(8),
    NN.flatten,
    NN.Dense(128, 1, AD.sigmoid)
)

Main.NN.Chain(Any[var"#11#13"(), Main.NN.Embedding(12849, 50, Float32[0.90951 -0.58014 … 0.014323 0.0; -0.20702 -1.1316 … -0.74624 0.0; … ; -0.60396 0.1722 … -0.079873 0.0; -0.31136 1.2622 … -0.89164 0.0], Int64[]), var"#12#14"(), Main.NN.Conv1D(3, 50, 8, Float32[-0.021721082 0.044999346 … -0.0002444767 -0.079383954; 0.07462665 -0.07178088 … 0.095087536 -0.13069274; 0.17371291 0.052133918 … 0.18355247 0.17799269;;; 0.083984345 -0.02572545 … 0.0016817462 0.027158981; -0.027506908 -0.061476417 … -0.09277021 -0.16816397; 0.18956283 -0.018067691 … -0.29008913 0.17985679;;; -0.110902116 -0.053898543 … -0.084604725 0.025174089; 0.0110642845 -0.0029188478 … -0.01599332 -0.14412504; -0.027440036 -0.16537227 … 0.12081164 -0.06867169;;; -0.020461172 -0.047929853 … -0.115899384 0.17314582; -0.053376526 -0.023527045 … -0.06317117 0.11143744; 0.060698405 -0.033579405 … -0.010276429 -0.017855657;;; 0.005213285 -0.0044138627 … -0.06694151 0.09043723; -0.002147967 -0.27466884 … 0.04673689 -0.005317950

In [5]:
accuracy(m, x, y) = mean((vec(m(x)) .> 0.5) .== (y .> 0.5))

accuracy (generic function with 1 method)

In [6]:
#= Trening modelu =#
opt = NN.Adam(Float32(0.001))
epochs = 5
NN.train_model(model, dataset, X_test, y_test, opt, epochs)

Epoch: 1 (56.46s) 	Train: (l: 0.59, a: 0.68) 	Test: (l: 0.50, a: 0.76) 	Memory allocated: 24791.629 MB
Epoch: 2 (77.01s) 	Train: (l: 0.48, a: 0.77) 	Test: (l: 0.46, a: 0.79) 	Memory allocated: 24372.825 MB
Epoch: 3 (48.84s) 	Train: (l: 0.44, a: 0.79) 	Test: (l: 0.43, a: 0.80) 	Memory allocated: 24380.843 MB
Epoch: 4 (45.93s) 	Train: (l: 0.42, a: 0.81) 	Test: (l: 0.41, a: 0.81) 	Memory allocated: 24385.158 MB
Epoch: 5 (47.60s) 	Train: (l: 0.39, a: 0.83) 	Test: (l: 0.39, a: 0.82) 	Memory allocated: 24388.209 MB
